In [20]:
import cv2     # for capturing videos
import math   # for mathematical operations
import matplotlib.pyplot as plt    # for plotting the images
%matplotlib inline
import pandas as pd
# from keras.preprocessing import image   # for preprocessing the images
import numpy as np    # for mathematical operations
# from keras.utils import np_utils
# from skimage.transform import resize   # for resizing images
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import train_test_split
import random as rnd
from glob import glob
from tqdm import tqdm

from PIL import Image
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision
from torchvision import transforms, datasets
device='cuda:0'

In [21]:
class ResNet(nn.Module):
    def __init__(self):
        super().__init__()
        
        self.convRe = nn.Conv3d(1, 180, (3,5,5))
        
        
        self.conv1 = nn.Conv3d(1, 180, (3,5,5))
        self.conv2 = nn.Conv3d(180, 180, (3,5,5))
        self.conv3 = nn.Conv3d(180, 180, (3,5,5))
        self.conv4 = nn.Conv3d(180, 180, (3,5,5))
        self.conv5 = nn.Conv3d(180, 36, (1,5,5))
        
        self.down1 = nn.Upsample((5,92,92),mode='trilinear')
        self.down2 = nn.Upsample((1,84,84),mode='trilinear')

        x= torch.randn(1,1,9,100,100)
        self._to_linear = None
        self.convs(x)
        
        self.fc1 = nn.Linear(self._to_linear, 700)
        self.fc2 = nn.Linear(700, 200)
        self.fc3 = nn.Linear(200, 101)        
        
        self.batch1 = nn.BatchNorm1d(self._to_linear)
        self.batch2 = nn.BatchNorm1d(750)
        self.batch3 = nn.BatchNorm1d(200)
        self.drop2 = nn.Dropout(p=0.2)
        
        
    def convs(self, x):
        inp1 = self.down1(x)
        x = F.relu(self.conv1(x))
        x = F.relu(self.conv2(x))
        inp2 = self.down2(x)
        x+=inp1
        x = F.relu(self.conv3(x))
        x = F.relu(self.conv4(x))
        x+=inp2
        x = F.relu(self.conv5(x))
        
        if self._to_linear is None:
            self._to_linear = x[0].shape[0]*x[0].shape[1]*x[0].shape[2]*x[0].shape[3]
            
        return x
    
    def forward(self, x):
        x = self.convs(x)
        x = x.view(-1, self._to_linear)
        x = F.leaky_relu(self.fc1(x))
        x = F.leaky_relu(self.fc2(x))
        x = self.batch3(x)
        x = self.drop2(x)
        x = self.fc3(x)
        return F.softmax(x, dim=1)

In [19]:
net = ResNet().to(device)
optimizer = optim.Adam(net.parameters(), lr=1e-5)
torch.cuda.memory_allocated()*1e-9

0.679337984

In [79]:
x  = torch.tensor([.5,.75,.875,.75,.5], dtype=torch.float32)
x = x.view(1,1,x.shape[0])
layer = nn.Conv1d(1, 2,(3),bias=False)
layer.weight = torch.nn.Parameter(torch.tensor(np.ones(layer.weight.shape),dtype=torch.float))
layer.requires_gradient=False
layer(x)/(torch.sum(x))

tensor([[[0.6296, 0.7037, 0.6296],
         [0.6296, 0.7037, 0.6296]]], grad_fn=<DivBackward0>)

In [7]:
x  = torch.tensor([1,2,3,4,3,2,1], dtype=torch.float32)
x = x.view(1,1,x.shape[0])
layer = nn.Conv1d(1, 2,(3), bias=False)
weights=np.ones(layer.weight.shape)
weights=weights/np.sum(weights)
layer.weight = torch.nn.Parameter(torch.tensor(weights,dtype=torch.float))
newX  = layer(x)
up = nn.Upsample(3,mode='linear')
print(newX)
up(newX)

tensor([[[1.0000, 1.5000, 1.6667, 1.5000, 1.0000],
         [1.0000, 1.5000, 1.6667, 1.5000, 1.0000]]],
       grad_fn=<SqueezeBackward1>)


tensor([[[1.1667, 1.6667, 1.1667],
         [1.1667, 1.6667, 1.1667]]], grad_fn=<UpsampleLinear1DBackward>)